# QPV2
QPV2 is an automatic quantum program verifier based on Python. Using a simple environment for definitions, it deals with the calculation and verification of quantum operators and programs. It helps users build operators with Dirac notations from the Python matrices, write valid quantum programs, conduct the classical simulataion and checks whether the program behaves as expected.


## Installation

After cloning the repository, navigate to its root folder and run
```
pip install -r requirements.txt
```
It will install all the dependent packages. Afterwards, test whether the software works well by running
```
pytest
```
If the all test are passed then the tool is well installed.

## Hello World Example

This is a simple example demonstrating the function and usage of QPV2. In the example, We refine the prescription $$[\mathrm{pre}: \ket{00}_{p, q} \bra{00}, \mathrm{post}: \ket{++}_{p, q}\bra{++}]$$ with program $prog$ and simulate the computing result.

In [12]:
from qpv2 import *
prover_restart()
prover(
    r'''
    Def HH := (H \otimes H).
    Def prog := Prog HH[p q].
    Refine pf : [pre : P0[p] \otimes P0[q], post : Pp[p] \otimes Pp[q]].
        Step proc prog.
    End.
    Def res := [[proc pf]](c1[]).
    '''
)

Here is the step-wise explanation.

First we import the package and reset the prover.

In [13]:
from qpv2 import *
prover_restart()
print(prover_info())


----------------------------------------
= Info (line 1) =

Emtpy Prover.



The prover has a variable environment, and commonly used quantum operators are predefined in the environment. Use `Show Def.` to print all the definitions.

In [14]:
prover(r"Show Def.")
print(prover_info())


----------------------------------------
= Info (line 1) =

Definitions: 
c1
c0
I
X
Y
Z
H
S
CX
CZ
CH
SWAP
CCX
Idiv2
Zero0
Zero
P0
P0div2
P1
P1div2
Pp
Ppdiv2
Pm
Pmdiv2
P00
Eq01_2
Neq01_2
Eq01_3
Omega
E10
E'P0
E'P1
E'DP
E'Set0




Use `Show <ID>.` to print the definition.

In [15]:
prover(r"Show CX.")
print(prover_info())


----------------------------------------
= Info (line 1) =

Show CX: 
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]



Define an operator.

In [16]:
prover(r"Def HH := (H \otimes H). Show HH.")
print(prover_info())


----------------------------------------
= Info (line 1) =

Show HH: 
(H ⊗ H)



Define the program $prog$ using operator $HH$.

In [17]:
prover(r'''
       Def prog := Prog 
            HH[p q]. 
       Show prog.
       ''')
print(prover_info())


----------------------------------------
= Info (line 1) =

Show prog: 
HH[p q]



Then we conduct a refinement. We need a program which transforms $\ket{00}$ state into $\ket{++}$ state, so the program prescription is:
$$
[\mathrm{pre}: \ket{00}_{p, q} \bra{00}, \mathrm{post}: \ket{++}_{p, q}\bra{++}].
$$

In [18]:
prover(r'''
    Refine pf : [pre : P0[p] \otimes P0[q], post : Pp[p] \otimes Pp[q]].
''')
print(prover_info())

----------------------------------------
= Refinement Mode =

Goal (1/1)
[ pre: (P0[p] ⊗ P0[q]), post: (Pp[p] ⊗ Pp[q]) ]

----------------------------------------
= Info (line 3) =

Refinement starts.



And as the information suggested, we entered the refinement mode. It quite resembles the proof mode in other interactive theorem provers, but here the goals become the prescriptions to be refined.

Actually, the program $prog$ we have defined satisfies this prescription. We then use it to complete the goal and finish the refinement.

prover(r'''
    Step proc prog.
    End.
''')
print(prover_info())

Calculate the classical simulation of $\llbracket prog \rrbracket (1)$. Note that here the density operator $1$ is automatically extended to the system of $[p\ q]$, with the assumption that their initial states are $\ket{0}$.

The result state is $\ket{++}_{p,q}$, exactly as expected.

In [20]:
prover(r"Def res := [[proc prog]](c1[]). Show res.")
print(prover_info())


----------------------------------------
= Info (line 6) =

Show res: 
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]][p q]



## Quantum Variables

In QPV2, the whole quantum system consists of qubits, and every qubit is denoted by an identifier (a string following the regular expression `[a-zA-Z\'][a-zA-Z\'0-9]*`). A quantum variable is an ordered list of unique qubit identifiers. For example, valid quantum variables include:
- `[]`, `[p]`, `[p q r']`, ...
  
And invalid quantum variables include:
- `[p p]`, `[1p 2q]`, ...

## Constructing Operators

QPV2 uses a natural grammar to encode Dirac notations and express quantum operators. In QPV2, operators are divided into two types: those without quantum variable indices (unlabelled) are denoted as $o$, and those indexed operators are denoted as $oi$.

The grammar for unlabelled quantum variable is:

$$
\begin{aligned}
    o ::=\ &C\ |\ - o\ |\ o + o\ |\ o - o\\
        & |\ c * o\ |\ c\ o\ \\
        & |\ o * o\ |\ o\ o\ |\ o^\dagger \\
        & |\ o \otimes o\ \\
        & |\ o \vee o\ |\ o \wedge o\ |\ o\ \^{} \bot \\
        & |\ o \rightsquigarrow o\ |\ o \Cap o.
\end{aligned}
$$

The grammar for labelled quantum variable is:
$$
\begin{aligned}
    oi ::= \ & \text{\texttt{IQOPT}}\ C\ |\ o\ qvar \\
        & |\ -oi\ |\ oi + oi\ |\ oi - oi\\
        & |\ c*oi\ |\ c\ oi\\
        & |\ oi * oi\ |\ oi\ oi\ |\ oi^\dagger\\
        & |\ oi \otimes oi\\
        & |\ oi \vee oi\ |\ oi \wedge oi\ |\ oi\ \^{}\bot\\
        & |\ oi \rightsquigarrow oi\ |\ oi \Cap oi.
\end{aligned}
$$

## Constructing Program

Quantum program statements, denoted as $stm$, are generated by the following grammar.
- $\texttt{abort}$
- $\texttt{skip}$
- $qvar \texttt{:=0}$
- $oi$
- $\texttt{assert}\ oi$
- $\texttt{[pre: } oi_1\texttt{, post: } oi_2 \texttt{]}$
- $ stm_1 \texttt{; }stm_2 $
- $ \texttt{(} stm_1\ p\ \oplus\ stm_2 \texttt{)}$
- $ \texttt{if}\ oi\ \texttt{then}\ stm_1\ \texttt{else}\ stm_0\ \texttt{end}$
- $ \texttt{while}\ oi\ \texttt{do}\ stm\ \texttt{end}$
- $ \texttt{proc}\ C$
- $ prec\ \texttt{==>}\ stm$

## QPV2 Language

The QPV2 tool is manipulated by a simple imperative language, which consists of a sequence of commands. Here is the introduction to the commands.

- $\texttt{Def}\ C\ \texttt{:=}\ o \texttt{.}$
- $\texttt{Def}\ C\ \texttt{:=}\ oi \texttt{.}$
- $\texttt{Def}\ C\ \texttt{:= [[}\ stm \texttt{]](} oi \texttt{).} $
- $\texttt{Def}\ C\ \texttt{:=}\ \texttt{Prog}\ stm \texttt{.}$
- $\texttt{Def}\ C\ \texttt{:=}\ \texttt{Extract}\ C \texttt{.}$

- $\texttt{Refine}\ C : pres \texttt{.}$
- $\texttt{Step}\ stm \texttt{.}$
- $\texttt{Step}\ \texttt{Seq}\ oi \texttt{.}$
- $\texttt{Step}\ \texttt{If}\ oi \texttt{.}$
- $\texttt{Step}\ \texttt{While}\ oi_1\ \texttt{Inv}\ oi_2 \texttt{.}$
- $\texttt{Choose}\ N \texttt{.}$
- $\texttt{End} \texttt{.}$

- $\texttt{Pause} \texttt{.}$
- $\texttt{Show}\ \texttt{Def} \texttt{.}$
- $\texttt{Show}\ C \texttt{.}$
- $\texttt{Eval}\ C \texttt{.}$
- $\texttt{Test}\ o_1\ \texttt{=}\ o_2 \texttt{.}$
- $\texttt{Test}\ o_1\ \texttt{<=}\ o_2 \texttt{.}$
- $\texttt{Test}\ oi_1\ \texttt{=}\ oi_2 \texttt{.}$
- $\texttt{Test}\ oi_1\ \texttt{<=}\ oi_2 \texttt{.}$

